https://en.wikipedia.org/wiki/Scale-invariant_feature_transform
https://kushalvyas.github.io/BOV.html
http://127.0.0.1:5500/image-feature-extraction.html
https://www.geeksforgeeks.org/feature-extraction-in-image-processing-techniques-and-applications/

### The main goal of this exercise is to get a feeling and understanding on the importance of representation and extraction of information from complex media content, in this case images. 

1. Start Simple with Colour Histograms
2. Explore Key Point-Based Feature Extraction
    - Explore Key Point-Based Feature Extraction: Once you’re comfortable with simple features, try using SIFT for key point detection and descriptor extraction. SIFT is robust to scale and rotation, which makes it ideal for finding distinctive features in images.
        - Visual Bag of Words: After extracting SIFT features, apply a bag-of-words approach to cluster these descriptors into visual “words.” This converts a variable number of key points into a fixed-length feature vector suitable for classifiers.

        

3. Then, try to use deep learning approaches, such as convolutional neural networks (for images) or recurrent neural networks (for text), or other approaches (but likely not just simple MLPs), and see
how your performance differs. Try at least two different architectures, they can be (reusing or be based on )existing, well-known ones.

Compare not just the overall measures, but perform a detailed comparison and analysis per class (confusion matrix), to identify if the two approaches lead to different types of errors in the different classes, and also try to identify other patterns.

For images, you can base your DL implementation on the tutorial provided by colleagues at the institute, available at https://github.com/tuwien-musicir/DL_Tutorial/blob/master/Car_recognition.ipynb (you can also check the rest of the repository for interesting code; credit to Thomas Lidy (http://www.ifs.tuwien.ac.at/~lidy/)). Mind also that you will find plenty of examples on how to create and train CNNs / RNNs in various frameworks - tensorflow, keras, pytorch, ....

Also perform a detailed comparison of runtime, considering both time for training and testing, including also the feature extraction components.
For the datasets you shall work with, pick two text/image datasets, from the list of suggestions below.

In [3]:
import pandas as pd
import numpy as np

In [5]:
import pandas as pd

splits = {'train': 'data/train-00000-of-00001-1359597a978bc4fa.parquet', 'valid': 'data/valid-00000-of-00001-70d52db3c749a935.parquet'}
df = pd.read_parquet("hf://datasets/zh-plus/tiny-imagenet/" + splits["train"])

In [ ]:
df.to_csv()